In [1]:
## This jupyter notebook is used to backstage the strategies with action pre-selection using historical and pre-saved data
import sys
sys.path.append("..")

import vectorbtpro as vbt
import numpy as np
import pandas as pd

import importlib
import inspect
from numba import njit
import talib
import math

from core import strat, indicators, presel

In [27]:
importlib.reload(indicators)
importlib.reload(strat)
importlib.reload(presel)


<module 'core.presel' from '/home/maxime/py-trading-bot6_StratObj/py-trading-bot/py-trading-bot/jupyter_scripts/../core/presel.py'>

In [28]:
period="2007_2023"
symbol_index="CAC40" # "CAC40", "DAX", "NASDAQ", "REALESTATE", "INDUSTRY", "IT", "COM", "CONSUMER", "HEALTHCARE"

In [29]:
#bti=presel.PreselVol(period,symbol_index=symbol_index)
#bti=presel.PreselMacdVol(period,symbol_index=symbol_index)
#bti=presel.PreselHistVol(period,symbol_index=symbol_index)
#bti=presel.PreselMacdVolMacro(period,symbol_index=symbol_index)
#bti=presel.PreselRetard(period,symbol_index=symbol_index)
#bti=presel.PreselRetardMacro(period,symbol_index=symbol_index)
#bti=presel.PreselDivergence(period,symbol_index=symbol_index)
#bti=presel.PreselDivergenceBlocked(period,symbol_index=symbol_index)

### Slow ###
#bti=presel.PreselVolSlow(period,symbol_index=symbol_index)
#bti=presel.PreselMacdVolSlow(period,symbol_index=symbol_index)
#bti=presel.PreselHistVolSlow(period,symbol_index=symbol_index)
#bti=presel.PreselRealMadrid(period,symbol_index=symbol_index)
#bti=presel.PreselRealMadridBlocked(period,symbol_index=symbol_index)

bti=presel.PreselWQ(period,symbol_index=symbol_index, nb=54)

bti.run()

  0%|          | 0/39 [00:00<?, ?it/s]

file:1: RuntimeWarning:

divide by zero encountered in divide

file:1: RuntimeWarning:

invalid value encountered in divide



In [30]:
pf=vbt.Portfolio.from_signals(bti.close, bti.entries,bti.exits,short_entries=bti.entries_short,short_exits  =bti.exits_short,freq="1d",
                            call_seq='auto',cash_sharing=True,
                             )
pf.returns_stats()

Start                        2007-01-02 00:00:00+00:00
End                          2022-12-30 00:00:00+00:00
Period                              4091 days 00:00:00
Total Return [%]                          87782.792731
Benchmark Return [%]                        313.602061
Annualized Return [%]                        52.732338
Annualized Volatility [%]                    39.787488
Max Drawdown [%]                             48.227022
Max Drawdown Duration                414 days 00:00:00
Sharpe Ratio                                  1.719342
Calmar Ratio                                  1.093419
Omega Ratio                                    1.30006
Sortino Ratio                                 2.677833
Skew                                          0.307711
Kurtosis                                      6.154872
Tail Ratio                                    1.160981
Common Sense Ratio                            1.773194
Value at Risk                                -0.029046
Alpha     

In [202]:
pf.stats()

Start                          2007-01-02 00:00:00+00:00
End                            2022-12-30 00:00:00+00:00
Period                                4091 days 00:00:00
Start Value                                        100.0
Min Value                                      81.420997
Max Value                                    1318.592013
End Value                                     896.624325
Total Return [%]                              796.624325
Benchmark Return [%]                          313.602061
Total Time Exposure [%]                        79.124908
Max Gross Exposure [%]                             100.0
Max Drawdown [%]                               47.488537
Max Drawdown Duration                  743 days 00:00:00
Total Orders                                         253
Total Fees Paid                                      0.0
Total Trades                                         127
Win Rate [%]                                        50.0
Best Trade [%]                 

In [22]:
pf.plot()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning: Subplot 'orders' does not support grouped data
  warnings.warn(warning_message)
/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/generic/plots_builder.py:398: UserWarning: Subplot 'trade_pnl' does not support grouped data
  warnings.warn(warning_message)


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#7f7f7f'},
              'mode': 'lines',
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'aa5c3528-8b86-4a59-98b0-cf1d577e1549',
              'x': array([datetime.datetime(2007, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 3, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2022, 12, 28, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 30, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1.        , 0.99987083, 0.99466387, ..., 4.150713

In [ ]:
symbol="KER"
sub_pf=vbt.Portfolio.from_signals(bti.close[symbol], bti.entries[symbol],bti.exits[symbol],short_entries=bti.entries_short[symbol],
                                  short_exits  =bti.exits_short[symbol],freq="1d",
                                  call_seq='auto')
sub_pf.returns_stats()

In [63]:
sub_pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'na…

In [47]:
pf.trades.records.sort_values(["entry_idx"], ascending=True).head(50)

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
0,0,0,4.878988,1,20.496056,0.0,31,24.435600,0.0,19.220989,0.192210,0,1,0
105,0,12,32.882844,31,3.625629,0.0,48,3.457366,0.0,-5.532972,-0.046409,0,1,0
162,0,17,2.098665,48,54.171600,0.0,58,54.853436,0.0,1.430943,0.012587,0,1,0
60,0,7,3.129873,58,36.780716,0.0,67,37.058296,0.0,0.868791,0.007547,0,1,0
115,0,13,3.418924,67,33.925224,0.0,71,34.276981,0.0,1.202630,0.010369,0,1,0
257,0,26,2.147585,71,54.568439,0.0,72,55.088955,0.0,1.117851,0.009539,0,1,0
243,0,25,5.467905,72,21.636850,0.0,76,21.952131,0.0,1.723926,0.014571,0,1,0
106,1,12,33.383826,78,3.595518,0.0,81,3.839058,0.0,8.130291,0.067734,0,1,1
137,0,15,10.826798,81,11.837521,0.0,83,12.187213,0.0,3.786049,0.029541,0,1,0
1,1,0,5.449387,83,24.213459,0.0,84,24.782698,0.0,3.102002,0.023509,0,1,1


In [15]:
### Statistics which action what ordered the most ###
pf.trades.records.groupby("col").count()

,id,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
col,,,,,,,,,,,,,
0,10,10,10,10,10,10,10,10,10,10,10,10,10
1,2,2,2,2,2,2,2,2,2,2,2,2,2
2,12,12,12,12,12,12,12,12,12,12,12,12,12
3,9,9,9,9,9,9,9,9,9,9,9,9,9
4,7,7,7,7,7,7,7,7,7,7,7,7,7
5,9,9,9,9,9,9,9,9,9,9,9,9,9
6,8,8,8,8,8,8,8,8,8,8,8,8,8
7,4,4,4,4,4,4,4,4,4,4,4,4,4
8,1,1,1,1,1,1,1,1,1,1,1,1,1


In [198]:
bti.macro_trend.vbt.plot()

FigureWidget({
    'data': [{'name': "(0.03, 'Close')",
              'showlegend': True,
              'type': 'scatter',
              'uid': '7c1dc43b-7774-4168-9137-e8580655c1b0',
              'x': array([datetime.datetime(2007, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 3, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2022, 12, 28, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 30, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([ 0,  0,  0, ..., -1, -1, -1])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                     